# LeNet CNN

In [2]:
import tensorflow as tf

In [3]:
# MNIST data from http://yann.lecun.com/exdb/mnist/
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
print(mnist.train.num_examples) # Number of training data
print(mnist.test.num_examples) # Number of test data

55000
10000


In [5]:
# architecture hyper-parameter
learning_rate = 0.01
training_iters = 100000
batch_size = 128
display_step = 20

n_input = 784 # 28x28 image
n_classes = 10 # 1 for each digit [0-9]
dropout = 0.75 

In [7]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)
print(x.shape, y.shape)

(?, 784) (?, 10)


In [8]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [9]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [10]:
def conv_net(x, weights, biases, dropout):
    # reshape input to 28x28 size
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max pooling
    conv1 = maxpool2d(conv1, k=2)

    # Convolution layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max pooling
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [11]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [12]:
# Create the model
model = conv_net(x, weights, biases, keep_prob)
print(model)
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_model = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_model, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

Tensor("Add_1:0", shape=(?, 10), dtype=float32)


In [13]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)    
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        if step % display_step == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Loss= " + \
                  "{:.3f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 2560, Loss= 1919.247, Training Accuracy= 0.680
Iter 5120, Loss= 668.470, Training Accuracy= 0.805
Iter 7680, Loss= 522.471, Training Accuracy= 0.844
Iter 10240, Loss= 292.444, Training Accuracy= 0.867
Iter 12800, Loss= 360.425, Training Accuracy= 0.844
Iter 15360, Loss= 163.273, Training Accuracy= 0.914
Iter 17920, Loss= 128.792, Training Accuracy= 0.906
Iter 20480, Loss= 44.306, Training Accuracy= 0.945
Iter 23040, Loss= 101.110, Training Accuracy= 0.953
Iter 25600, Loss= 80.861, Training Accuracy= 0.914
Iter 28160, Loss= 106.317, Training Accuracy= 0.930
Iter 30720, Loss= 84.872, Training Accuracy= 0.938
Iter 33280, Loss= 49.033, Training Accuracy= 0.953
Iter 35840, Loss= 76.170, Training Accuracy= 0.961
Iter 38400, Loss= 9.664, Training Accuracy= 0.977
Iter 40960, Loss= 136.264, Training Accuracy= 0.914
Iter 43520, Loss= 67.540, Training Accuracy= 0.922
Iter 46080, Loss= 14.147, Training Accuracy= 0.977
Iter 48640, Loss= 85.346, Training Accuracy= 0.938
Iter 51200, Loss= 56.480

In [ ]:
'''
In-class activity:
1. Add one more layer after conv2. The new layer will also have kernel of size 5 x 5, 128 of them.
2. The resultant output of the filter will be a 4*4 image.
3. Add bias as well with a length of 128. 
4. Rerun the architecture.
'''